# kospi 데이터 탐색
> 문제 정의에 대한 이해를 위해 target 변수(코스피지수, Y)가 있는 kospi_국면분석.csv의 데이터를 탐색하는 소스코드 입니다.

## 준비하기

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from encodings import euc_kr

## kospi 불러오기 

In [92]:
kospi = pd.read_csv('./KOSPI_국면분석.csv',encoding = 'euc-kr')

In [4]:
kospi.head()

,날짜,Y,지수시가,지수고가,지수저가,지수종가,거래량,거래대금,상장주식수,시가총액,...,보합종목수,상한종목수,하한종목수,연중최고가종목수,연중최저가종목수,25일이평상회종목건수,25일이평하회종목건수,52주신고가종목수,52주신저가종목수,배당 수익율
0,2001-01-02,상승,503.31,521.34,500.97,520.95,231014600,1223328,19638887,191252891,...,22,167,16,0,0,26,73,1,85,0.0
1,2001-01-03,상승,512.74,524.58,510.70,521.43,324584400,1583403,19633559,191363161,...,52,85,6,0,0,28,71,1,12,0.0
2,2001-01-04,상승,551.53,567.16,550.91,558.02,444540900,2984761,19634734,204074166,...,20,149,3,0,0,42,57,1,2,0.0
3,2001-01-05,상승,559.54,581.41,555.40,580.85,578286000,3569582,19637038,212089180,...,37,92,9,0,0,43,56,3,2,0.0
4,2001-01-08,상승,573.72,587.91,572.48,586.65,558645100,2948092,19630735,214231273,...,45,72,5,0,0,43,56,6,1,0.0


> 코스피 지수를 나타내는 칼럼이 총 4가지 있다. -> 지수시가, 지수고가, 지수저가, 지수종가  (*배당 수익율, 연중최고가종목수, 연중최저가종목수의 값이 0으로 처리된 것을 확인할 수 있다*)

In [5]:
kospi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4449 entries, 0 to 4448
Data columns (total 31 columns):
날짜             4449 non-null object
Y              4449 non-null object
지수시가           4449 non-null float64
지수고가           4449 non-null float64
지수저가           4449 non-null float64
지수종가           4449 non-null float64
거래량            4449 non-null int64
거래대금           4449 non-null int64
상장주식수          4449 non-null int64
시가총액           4449 non-null int64
자본금            4449 non-null int64
외국인보유주식수       4449 non-null int64
외국인보유시가총액      4449 non-null int64
신용거래종목수        4449 non-null int64
신용가능종목거래량      4449 non-null int64
신용자료일자         4448 non-null float64
전체종목수          4449 non-null int64
회사수            4449 non-null int64
거래형성종목수        4449 non-null int64
상승종목수          4449 non-null int64
하락종목수          4449 non-null int64
보합종목수          4449 non-null int64
상한종목수          4449 non-null int64
하한종목수          4449 non-null int64
연중최고가종목수       4449 non-null int64
연중최저가종

> '**날짜**'와 '**Y**'를 제외한 변수들의 type은 모두 numeric이다.

## Y의 '상승'은 (기준일의 지수종가 - 5일 후의 지수종가) < 0 일 때를 의미하는가?

#### diff: 5일 후와 비교하기 - diff_5()

In [109]:
## diff 계산 모듈 
def diff(x,col):
    # after5: 5일 후 지수종가
    after5 = x[[col]].shift(-5)
    
    # diff: t시점의 지수종가 - t+5시점의 지수종가
    diff = x[[col]] - after5 #5일 후 지수종가와의 차이
    
    diff.columns = ['diff']
    
    return diff

In [110]:
diff = diff(kospi,'지수종가')
diff

,diff
0,-68.97
1,-39.38
2,-3.77
3,-7.02
4,-12.35
5,-13.50
6,-35.02
7,-42.26
8,-31.91
9,-28.45


#### diff_tf: diff의 부호에 대한 논리연산

In [111]:
# diff_tf = diff>0 인지에 대한 TF
diff_tf = diff[['diff']]<0 #True면 상승 #False면 하락

## column 이름 변경 'diff'->'Y_TF': Y를 True/False으로 나타낸 변수라서
diff_tf

,diff
0,True
1,True
2,True
3,True
4,True
5,True
6,True
7,True
8,True
9,True


In [112]:
## True/False -> 상승/하락 으로 변경하기
for i in range(diff_tf.shape[0]):
        if diff_tf.iloc[i,0]==True:
            diff_tf.iloc[i,0] = '상승'
        else:
            diff_tf.iloc[i,0] = '하락'
diff_tf.shape

(4449, 1)

#### diff_tf와 Y 비교하기

In [113]:
# 확인하기
## Y_TF ==  Y 여부
y = kospi[['Y']]
for i in range(diff_tf.shape[0]):
    if y.iloc[i,0]!=diff_tf.iloc[i,0]:
        print(i,False)
        print(y.iloc[i,0],diff_tf.iloc[i,0])
#answer = pd.concat([kospi[['Y']],diff_tf],axis = 1)
#answer
# 4444~4448은 NaN값 = False값
# 없음

4447 False
상승 하락


- 데이터 탐색 결과, Y가 뜻하는 '상승'은 **t+5시점의 지수종가>t시점의 지수종가**를 의미하고 '하락'은 **t+5시점의 지수종가 <= t시점의 지수종가**를 의미한다.
- 따라서 Y를 결정짓는 칼럼은 '**지수종가**'이므로 이에 따른 분석 방향을 결정한다.